### Imports

In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

### Load all dataframes - win percentages, stadium and conference info, and attendance info

In [2]:
path = '/Users/murdock/Documents/PDF_files/'
win_perc_df = pd.read_pickle(path + 'win_perc_df.pkl')
stadiums_conf_df = pd.read_pickle(path + 'stadiums_conf_df.pkl')
attendance_df = pd.read_pickle(path + 'attendance_df.pkl')

In [3]:
college_enrollment_df = pd.read_csv('/Users/murdock/Desktop/ncaa_college_enrollment.csv', skiprows=[0, 1])

### Start working with win percentages for combining dataframes

In [4]:
win_perc_df.head()

,TEAMS,WIN_PERCENT,YEARS
0,Clemson,0.642857,2009
1,Boston College,0.615385,2009
2,Florida State,0.538462,2009
3,Wake Forest,0.416667,2009
4,NC State,0.416667,2009


### Load values only from 2009-2015 (drop 2016 because enrollment data not available)

In [5]:
win_perc_df = win_perc_df[win_perc_df['YEARS'] != '2016']

In [6]:
win_perc_df.tail()

,TEAMS,WIN_PERCENT,YEARS
1860,Grambling State,0.750000,2015
1861,Prairie View A,0.800000,2015
1862,Southern,0.545455,2015
1863,Texas Southern,0.300000,2015
1864,Arkansas-Pine,0.181818,2015


In [ ]:
stadiums_conf_df.head()

In [ ]:
attendance_df.head()

In [7]:
attendance_df = attendance_df[attendance_df['Year'] != '2016']

In [8]:
attendance_df[attendance_df['School'] == 'alcorn']

,School,Home_Games,Attendance,Average,Year
846,alcorn,6,"73,446","12,241",2012
1091,alcorn,5,"20,649","4,130",2013
1340,alcorn,7,"85,873","12,268",2014
1587,alcorn,5,"61,716","12,343",2015


In [9]:
college_enrollment_df[college_enrollment_df['Institution Name'].str.contains('Mass')]

,UnitId,Institution Name,Fall 2003,Fall 2004,Fall 2005,Fall 2006,Fall 2007,Fall 2008,Fall 2009,Fall 2010,Fall 2011,Fall 2012,Fall 2013,Fall 2014,Fall 2015,Unnamed: 15
347,166513,University of Massachusetts-Lowell,11706,11089,10666,11635,11635,12471,13602,14702,15431,16294,16930,17179,18047,NaN
348,166629,University of Massachusetts-Amherst,24310,24646,25093,25873,25873,26359,27016,27569,28084,28236,28518,28635,29269,NaN
349,166638,University of Massachusetts-Boston,12394,11682,11862,13433,13433,14117,14912,15454,15741,15874,16277,16756,17030,NaN
350,166683,Massachusetts Institute of Technology,10340,10320,10206,10220,10220,10299,10384,10566,10894,11189,11301,11319,11331,NaN
351,166692,Massachusetts Maritime Academy,937,953,1008,1135,1135,1166,1292,1296,1368,1415,1476,1497,1674,NaN
357,167288,Massachusetts College of Liberal Arts,1811,1831,1849,1841,1841,1942,1962,1974,1886,1799,1716,1765,1641,NaN
366,167987,University of Massachusetts-Dartmouth,8284,8299,8549,9080,9080,9155,9302,9432,9225,9210,9053,9111,8916,NaN


### Use fuzzywuzzy to match schools and merge stadium info with win perc info

First clean up some names to make matching easier

In [10]:
stadiums_conf_df['TEAM'] = stadiums_conf_df['TEAM'].replace('USF', 'South Florida')
stadiums_conf_df['TEAM'] = stadiums_conf_df['TEAM'].replace('NIU', 'Northern Illinois')
stadiums_conf_df['TEAM'] = stadiums_conf_df['TEAM'].replace('FIU', 'Florida Intl')
win_perc_df['TEAMS'] = win_perc_df['TEAMS'].replace('USCUSC', 'South Carolina')
win_perc_df = win_perc_df[win_perc_df['TEAMS'] != 'UT San Antonio']

In [11]:
stadiums_conf_df[stadiums_conf_df['TEAM'] == 'Northern Illinois']

,TEAM,CONFERENCE,CAPACITY
144,Northern Illinois,MAC,23595


Now merge dataframes by adding a capacity and conference column to the win percentage dataframe

In [12]:
team_selection = stadiums_conf_df['TEAM']
capacities = []
conferences = []
for team in win_perc_df['TEAMS']:
    select = process.extractOne(team, team_selection, scorer=fuzz.ratio)
    print(team, select)
    capacity = stadiums_conf_df[stadiums_conf_df['TEAM'] == select[0]]['CAPACITY'].values
    conf = stadiums_conf_df[stadiums_conf_df['TEAM'] == select[0]]['CONFERENCE'].values
    #print(capacity[0], conf[0])
    capacities.append(capacity[0])
    conferences.append(conf[0])

win_perc_df['CAPACITY'] = capacities
win_perc_df['CONFERENCE'] = conferences
    

Clemson ('Clemson', 100, 168)
Boston College ('Boston College', 100, 130)
Florida State ('Florida State', 100, 141)
Wake Forest ('Wake Forest', 100, 136)
NC State ('NC State', 100, 251)
Maryland ('Maryland', 100, 152)
Georgia Tech ('Georgia Tech', 100, 142)
Virginia Tech ('Virginia Tech', 100, 194)
Miami ('Miami', 100, 173)
North Carolina ('North Carolina', 100, 188)
Duke ('Duke', 100, 145)
Virginia ('Virginia', 100, 153)
Nebraska ('Nebraska', 100, 207)
Missouri ('Missouri', 100, 165)
Kansas State ('Kansas State', 100, 139)
Iowa State ('Iowa State', 100, 178)
Colorado ('Colorado', 100, 166)
Kansas ('Kansas', 100, 206)
Texas ('Texas', 100, 158)
Oklahoma State ('Oklahoma State', 100, 143)
Texas Tech ('Texas Tech', 100, 185)
Oklahoma ('Oklahoma', 100, 170)
Texas A&M ('Texas A&M', 100, 192)
Baylor ('Baylor', 100, 203)
Cincinnati ('Cincinnati', 100, 213)
Pittsburgh ('Pittsburgh', 100, 174)
West Virginia ('West Virginia', 100, 210)
Rutgers ('Rutgers', 100, 175)
Connecticut ('Connecticut', 10

Boise State ('Boise State', 100, 127)
Wyoming ('Wyoming', 100, 184)
San Diego State ('San Diego State', 100, 234)
Air Force ('Air Force', 100, 163)
Colorado State ('Colorado State', 100, 236)
UNLVUNLV ('UNLV', 67, 231)
New Mexico ('New Mexico', 100, 162)
Oregon ('Oregon', 100, 134)
Stanford ('Stanford', 100, 239)
Washington ('Washington', 100, 126)
California ('California', 100, 150)
Oregon State ('Oregon State', 100, 223)
Washington State ('Washington State', 100, 201)
South Carolina ('South Carolina', 100, 253)
UCLAUCLA ('UCLA', 67, 227)
Arizona State ('Arizona State', 100, 242)
Utah ('Utah', 100, 226)
Colorado ('Colorado', 100, 166)
Arizona ('Arizona', 100, 133)
Georgia ('Georgia', 100, 232)
South Carolina ('South Carolina', 100, 253)
Florida ('Florida', 100, 240)
Vanderbilt ('Vanderbilt', 100, 246)
Kentucky ('Kentucky', 100, 191)
Tennessee ('Tennessee', 100, 212)
LSULSU ('LSU', 67, 244)
Alabama ('Alabama', 100, 147)
Arkansas ('Arkansas', 100, 222)
Auburn ('Auburn', 100, 218)
Missis

Ole Miss ('Southern Miss', 67, 154)
Arkansas ('Arkansas', 100, 222)
Louisiana ('Louisiana Tech', 78, 181)
Arkansas State ('Arkansas State', 100, 157)
Western Kentucky ('Western Kentucky', 100, 176)
Louisiana Monroe ('Louisiana-Monroe', 100, 186)
South Alabama ('South Alabama', 100, 193)
Troy ('Troy', 100, 248)
Texas State ('Texas State', 100, 179)
Georgia State ('Georgia State', 100, 217)
Memphis ('Memphis', 100, 197)
Cincinnati ('Cincinnati', 100, 213)
UCFUCF ('UCF', 67, 238)
East Carolina ('East Carolina', 100, 135)
Houston ('Houston', 100, 182)
Temple ('Temple', 100, 198)
South Florida ('South Florida', 100, 221)
Tulsa ('Tulsa', 100, 235)
Tulane ('Tulane', 100, 138)
SMUSMU ('SMU', 67, 171)
Connecticut ('Connecticut', 100, 220)
Florida State ('Florida State', 100, 141)
Clemson ('Clemson', 100, 168)
Louisville ('Louisville', 100, 216)
Boston College ('Boston College', 100, 130)
NC State ('NC State', 100, 251)
Syracuse ('Syracuse', 100, 155)
Wake Forest ('Wake Forest', 100, 136)
Georgi

Colorado ('Colorado', 100, 166)
Florida ('Florida', 100, 240)
Tennessee ('Tennessee', 100, 212)
Georgia ('Georgia', 100, 232)
Vanderbilt ('Vanderbilt', 100, 246)
Kentucky ('Kentucky', 100, 191)
Missouri ('Missouri', 100, 165)
South Carolina ('South Carolina', 100, 253)
Alabama ('Alabama', 100, 147)
Ole Miss ('Southern Miss', 67, 154)
Arkansas ('Arkansas', 100, 222)
LSULSU ('LSU', 67, 244)
Texas A&M ('Texas A&M', 100, 192)
Mississippi State ('Mississippi State', 100, 159)
Auburn ('Auburn', 100, 218)
Arkansas State ('Arkansas State', 100, 157)
Appalachian State ('Appalachian State', 100, 189)
Georgia Southern ('Georgia Southern', 100, 128)
Georgia State ('Georgia State', 100, 217)
South Alabama ('South Alabama', 100, 193)
Troy ('Troy', 100, 248)
New Mexico State ('New Mexico State', 100, 124)
Idaho ('Idaho', 100, 252)
Louisiana ('Louisiana Tech', 78, 181)
Texas State ('Texas State', 100, 179)
Louisiana Monroe ('Louisiana-Monroe', 100, 186)
Montana ('Montana Grizzlies', 58, 115)
Eastern W

North Carolina Central ('North Carolina Central Eagles', 86, 87)
Howard ('Howard Bison', 67, 120)
Northern Iowa ('Northern Iowa Panthers', 74, 111)
Western Illinois ('Northern Illinois', 79, 144)
North Dakota State ('North Dakota State Bison', 86, 43)
Illinois State ('Illinois State Redbirds', 76, 55)
Indiana State ('Michigan State', 74, 237)
Missouri State ('Missouri State Bears', 82, 91)
South Dakota State ('South Dakota Coyotes', 79, 32)
Southern Illinois ('Northern Illinois', 88, 144)
Youngstown State ('Youngstown State Penguins', 78, 105)
Central Connecticut ('Central Connecticut Blue Devils', 76, 12)
Robert Morris ('Robert Morris Colonials', 72, 67)
Duquesne ('Duquesne Dukes', 73, 11)
Bryant ('Bryant Bulldogs', 57, 24)
Albany ('Alabama', 62, 147)
Wagner ('Wagner Seahawks', 57, 113)
Monmouth ('Monmouth Hawks', 73, 72)
Sacred Heart ('Sacred Heart Pioneers', 73, 26)
St Francis ('Saint Francis Red Flash', 61, 34)
Southeast Missouri State ('Southeast Missouri State Redhawks', 84, 62)


Coastal Carolina ('Coastal Carolina', 100, 146)
Liberty ('Liberty Flames', 67, 121)
Charleston Southern ('Charleston Southern Buccaneers', 78, 23)
Gardner-Webb ("Gardner–Webb Runnin' Bulldogs", 59, 41)
VMIVMI ('Miami', 73, 173)
Presbyterian College ('Presbyterian Blue Hose', 76, 13)
Old Dominion ('Old Dominion', 100, 167)
New Hampshire ('New Hampshire Wildcats', 74, 117)
Richmond ('Richmond Spiders', 67, 37)
Villanova ('Villanova Wildcats', 67, 112)
Towson ('Towson Tigers', 63, 68)
James Madison ('James Madison Dukes', 81, 20)
Maine ('Miami', 60, 173)
Delaware ('Delaware State Hornets', 53, 7)
William ('Illinois', 53, 204)
Georgia State ('Georgia State', 100, 217)
Rhode Island ('Rhode Island Rams', 83, 77)
Penn ('Penn State', 57, 137)
Harvard ('Harvard Crimson', 64, 58)
Brown ('Brown Bears', 62, 22)
Dartmouth ('Dartmouth Big Green', 64, 78)
Princeton ('Princeton Tigers', 72, 92)
Cornell ('Cornell Big Red', 64, 102)
Columbia ('Columbia Lions', 73, 73)
Yale ('Syracuse', 50, 155)
Bethune-

Yale ('Syracuse', 50, 155)
Princeton ('Princeton Tigers', 72, 92)
Brown ('Brown Bears', 62, 22)
Penn ('Penn State', 57, 137)
Cornell ('Cornell Big Red', 64, 102)
Columbia ('Columbia Lions', 73, 73)
Morgan State ('Oregon State', 83, 223)
North Carolina Central ('North Carolina Central Eagles', 86, 87)
South Carolina State ('South Carolina State Bulldogs', 82, 88)
Bethune-Cookman ('Bethune–Cookman Wildcats', 77, 82)
North Carolina A ('North Carolina', 93, 188)
Norfolk State ('Norfolk State Spartans', 74, 118)
Florida A&M ('Florida', 78, 240)
Howard ('Howard Bison', 67, 120)
Hampton ('Hampton Pirates', 64, 10)
Delaware State ('Delaware State Hornets', 78, 7)
Savannah State ('Savannah State Tigers', 80, 108)
North Dakota State ('North Dakota State Bison', 86, 43)
Illinois State ('Illinois State Redbirds', 76, 55)
Northern Iowa ('Northern Iowa Panthers', 74, 111)
South Dakota State ('South Dakota Coyotes', 79, 32)
Indiana State ('Michigan State', 74, 237)
Youngstown State ('Youngstown State

In [13]:
win_perc_df.tail()

,TEAMS,WIN_PERCENT,YEARS,CAPACITY,CONFERENCE
1860,Grambling State,0.750000,2015,19600,SWAC
1861,Prairie View A,0.800000,2015,15000,SWAC
1862,Southern,0.545455,2015,36000,C-USA
1863,Texas Southern,0.300000,2015,22039,SWAC
1864,Arkansas-Pine,0.181818,2015,72000,SEC


In [ ]:
# Notes on fixes
# south florida - florida = exists as USF
# northern illinois - southern illinois salukis = exists as NIU
# uscusc - syracuse = south carolina
# florida intl - florida = exists FIU

### Merge attendance info with win perc df using fuzzywuzzy

Make some initial corrections

In [14]:
win_perc_df['TEAMS'] = win_perc_df['TEAMS'].replace('assachusetts', 'Massachusetts')
win_perc_df['TEAMS'] = win_perc_df['TEAMS'].replace('NC State', 'North Carolina St')
attendance_df = attendance_df[attendance_df['Attendance'] != 'UT San Antonio']

In [15]:
win_perc_df.head()

,TEAMS,WIN_PERCENT,YEARS,CAPACITY,CONFERENCE
0,Clemson,0.642857,2009,81500,ACC
1,Boston College,0.615385,2009,44500,ACC
2,Florida State,0.538462,2009,79560,ACC
3,Wake Forest,0.416667,2009,31500,ACC
4,North Carolina St,0.416667,2009,57583,ACC


In [16]:
attendance_df[attendance_df['School'] == 'south_alabama']

,School,Home_Games,Attendance,Average,Year
688,south_alabama,6,"110,654","18,442",2011


In [17]:
team_selection2 = attendance_df['School']
average_attendances = []
y = win_perc_df['YEARS'].values
t = win_perc_df['TEAMS'].values
combo = list(zip(t, y))

for team, year in combo:
    select = process.extractOne(team, team_selection2, scorer=fuzz.ratio)
    #print(team, select)
    attendance = attendance_df[(attendance_df['School'] == select[0]) & (attendance_df['Year'] == year)]['Average'].values
    if attendance.any():
        attend = attendance[0]
        average_attendances.append(attend.replace(',',''))
    else:
        attend = 'None'
        average_attendances.append(attend)

win_perc_df['AVG_ATTENDANCE'] = average_attendances

In [18]:
win_perc_df.head()

,TEAMS,WIN_PERCENT,YEARS,CAPACITY,CONFERENCE,AVG_ATTENDANCE
0,Clemson,0.642857,2009,81500,ACC,75793
1,Boston College,0.615385,2009,44500,ACC,35716
2,Florida State,0.538462,2009,79560,ACC,74345
3,Wake Forest,0.416667,2009,31500,ACC,31791
4,North Carolina St,0.416667,2009,57583,ACC,56422


In [19]:
combined_df = win_perc_df

In [20]:
combined_df.head()

,TEAMS,WIN_PERCENT,YEARS,CAPACITY,CONFERENCE,AVG_ATTENDANCE
0,Clemson,0.642857,2009,81500,ACC,75793
1,Boston College,0.615385,2009,44500,ACC,35716
2,Florida State,0.538462,2009,79560,ACC,74345
3,Wake Forest,0.416667,2009,31500,ACC,31791
4,North Carolina St,0.416667,2009,57583,ACC,56422


Pickle file to save progress

In [ ]:
finaldf_path = '/Users/murdock/Documents/PDF_files/'
combined_df.to_pickle(finaldf_path+ 'final_df.pkl')

### Merge final data, student enrollment, with dataframe

In [21]:
# Check fuzzywuzzy for matching schools. Different scoring did not produce great results. 
institutions = college_enrollment_df['Institution Name'].sort_values(ascending=False)
for team in combined_df['TEAMS'].values:
    institution = process.extractOne(team, institutions, scorer=fuzz.token_set_ratio)
    print(team, institution)

Clemson ('Clemson University', 100, 809)
Boston College ('Boston College', 100, 330)
Florida State ('Florida State University', 100, 141)
Wake Forest ('Wake Forest University', 100, 644)
North Carolina St ('University of North Carolina at Pembroke', 90, 639)
Maryland ('University of Maryland-College Park', 100, 310)
Georgia Tech ('University of West Georgia', 74, 176)
Virginia Tech ('West Virginia Wesleyan College', 76, 975)
Miami ('University of Miami', 100, 144)
North Carolina ('University of North Carolina at Pembroke', 100, 639)
Duke ('Duke University', 100, 612)
Virginia ('West Virginia Wesleyan College', 100, 975)
Nebraska ('University of Nebraska-Lincoln', 100, 467)
Missouri ('University of Missouri-St Louis', 100, 448)
Kansas State ('Kansas State University', 100, 262)
Iowa State ('Iowa State University', 100, 251)
Colorado ('Western State Colorado University', 100, 99)
Kansas ('University of Missouri-Kansas City', 100, 446)
Texas ('West Texas A & M University', 100, 897)
Oklah

Northwestern ('Northwestern University', 100, 213)
Purdue ('Purdue University-Main Campus', 100, 1012)
Minnesota ('University of Minnesota-Twin Cities', 100, 413)
Indiana ('University of Southern Indiana', 100, 233)
UCFUCF ('Utica College', 32, 600)
East Carolina ('East Carolina University', 100, 613)
Southern Mississippi ('University of Southern Mississippi', 100, 437)
Marshall ('Marshall University', 100, 969)
UABUAB ('Washburn University', 32, 264)
Memphis ('University of Memphis', 100, 841)
SMUSMU ('Muskingum University', 38, 673)
Tulsa ('University of Tulsa', 100, 698)
Houston ('University of Houston', 100, 863)
UTEPUTEP ('Temple University', 40, 784)
Rice ('Rice University', 100, 873)
Tulane ('Tulane University of Louisiana', 100, 291)
Navy ('SUNY Oneonta', 38, 590)
Notre Dame ('University of Notre Dame', 100, 237)
Army ('Marymount University', 33, 930)
Miami Oh ('University of Miami', 77, 144)
Ohio ('Ohio Wesleyan University', 100, 678)
Temple ('Temple University', 100, 784)
Ken

Arizona ('University of Arizona', 100, 20)
Georgia ('University of West Georgia', 100, 176)
South Carolina ('University of South Carolina-Upstate', 100, 824)
Florida ('University of South Florida-Main Campus', 100, 150)
Vanderbilt ('Vanderbilt University', 100, 851)
Kentucky ('Western Kentucky University', 100, 277)
Tennessee ('The University of Tennessee-Martin', 100, 847)
LSULSU ('Tusculum College', 45, 850)
Alabama ('University of West Alabama', 100, 9)
Arkansas ('University of Central Arkansas', 100, 29)
Auburn ('Auburn University', 100, 5)
Mississippi State ('Mississippi Valley State University', 100, 433)
Ole Miss ('Canisius College', 58, 518)
Arkansas State ('Arkansas State University-Main Campus', 100, 26)
Western Kentucky ('Western Kentucky University', 100, 277)
Louisiana ('University of Louisiana at Monroe', 100, 286)
Florida Intl ('University of South Florida-Main Campus', 74, 150)
North Texas ('University of North Texas', 100, 870)
Louisiana Monroe ('University of Louisian

North Carolina St ('University of North Carolina at Pembroke', 90, 639)
Duke ('Duke University', 100, 612)
Virginia Tech ('West Virginia Wesleyan College', 76, 975)
Miami ('University of Miami', 100, 144)
Georgia Tech ('University of West Georgia', 74, 176)
North Carolina ('University of North Carolina at Pembroke', 100, 639)
Pittsburgh ('University of Pittsburgh-Pittsburgh Campus', 100, 774)
Virginia ('West Virginia Wesleyan College', 100, 975)
Baylor ('University of Mary Hardin-Baylor', 100, 867)
Oklahoma ('University of Oklahoma-Norman Campus', 100, 694)
Oklahoma State ('Southwestern Oklahoma State University', 100, 697)
Texas ('West Texas A & M University', 100, 897)
Kansas State ('Kansas State University', 100, 262)
Texas Tech ('Texas Tech University', 100, 894)
TCUTCU ('Utica College', 42, 600)
West Virginia ('West Virginia Wesleyan College', 100, 975)
Iowa State ('Iowa State University', 100, 251)
Kansas ('University of Missouri-Kansas City', 100, 446)
Michigan State ('Michigan 

Southern Mississippi ('University of Southern Mississippi', 100, 437)
Notre Dame ('University of Notre Dame', 100, 237)
Navy ('SUNY Oneonta', 38, 590)
BYUBYU ('SUNY at Albany', 40, 579)
Army ('Marymount University', 33, 930)
Bowling Green ('Bowling Green State University-Main Campus', 100, 654)
Ohio ('Ohio Wesleyan University', 100, 678)
Buffalo ('University at Buffalo', 100, 581)
Akron ('University of Akron Main Campus', 100, 650)
UMass ('Furman University', 36, 814)
Miami Oh ('University of Miami', 77, 144)
Kent State ('Kent State University at Kent', 100, 666)
Northern Illinois ('Northern Illinois University', 100, 212)
Toledo ('University of Toledo', 100, 681)
Western Michigan ('Western Michigan University', 100, 401)
Central Michigan ('Central Michigan University', 100, 383)
Ball State ('Ball State University', 100, 223)
Eastern Michigan ('Eastern Michigan University', 100, 385)
Boise State ('Boise State University', 100, 181)
Colorado State ('Western State Colorado University', 1

...

Idaho State ('Idaho State University', 100, 182)
Liberty ('West Liberty University', 100, 974)
Coastal Carolina ('Coastal Carolina University', 100, 822)
Stony Brook ('Stony Brook University', 100, 582)
Gardner-Webb ('Gardner-Webb University', 100, 617)
VMIVMI ('Emory University', 36, 160)
Charleston Southern ('Charleston Southern University', 100, 805)
Presbyterian College ('Presbyterian College', 100, 819)
William ('William Peace University', 100, 638)
Delaware ('University of Delaware', 100, 119)
New Hampshire ('University of New Hampshire-Main Campus', 100, 477)
Villanova ('Villanova University', 100, 787)
UMass ('Furman University', 36, 814)
Rhode Island ('University of Rhode Island', 100, 801)
Richmond ('University of Richmond', 100, 936)
Maine ('University of Southern Maine', 100, 300)
James Madison ('James Madison University', 100, 924)
Towson ('Towson University', 100, 317)
Old Dominion ('Old Dominion University', 100, 932)
Georgia State ('Georgia State University', 100, 165)


Howard ('Howard University', 100, 129)
Morgan State ('Morgan State University', 100, 312)
North Carolina A ('North Carolina A & T State University', 100, 629)
Delaware State ('Delaware State University', 100, 118)
North Carolina Central ('North Carolina Central University', 100, 634)
Savannah State ('Savannah State University', 100, 173)
North Dakota State ('North Dakota State University-Main Campus', 100, 649)
Northern Iowa ('University of Northern Iowa', 100, 255)
Illinois State ('Illinois State University', 100, 202)
Indiana State ('Indiana State University', 100, 234)
South Dakota State ('South Dakota State University', 100, 829)
Youngstown State ('Youngstown State University', 100, 687)
Southern Illinois ('Southern Illinois University-Edwardsville', 100, 219)
Missouri State ('Southeast Missouri State University', 100, 454)
Western Illinois ('Western Illinois University', 100, 220)
Duquesne ('Duquesne University', 100, 729)
Albany ('SUNY at Albany', 100, 579)
Bryant ('Bryant Univer

Fordham ('Fordham University', 100, 538)
Bucknell ('Bucknell University', 100, 717)
Butler ('Butler University', 100, 224)
Drake ('Drake University', 100, 248)
San Diego ('University of San Diego', 100, 74)
Jacksonville ('Jacksonville University', 100, 143)
Dayton ('University of Dayton', 100, 659)
Marist ('Marist College', 100, 552)
Morehead State ('Wayne State College', 61, 468)
Davidson ('Davidson College', 100, 611)
Valparaiso ('Valparaiso University', 100, 242)
Campbell ('Campbell University', 100, 608)
Georgia Southern ('Georgia Southern University', 100, 164)
Wofford ('Wofford College', 100, 826)
Appalachian State ('Appalachian State University', 100, 605)
Chattanooga ('The University of Tennessee-Chattanooga', 100, 845)
Samford ('Samford University', 100, 13)
The Citadel ('Citadel Military College of South Carolina', 78, 808)
Furman ('Furman University', 100, 814)
Elon ('Elon University', 100, 615)
Western Carolina ('Western Carolina University', 100, 647)
Central Arkansas ('Un

Texas Southern ('Texas Southern University', 100, 893)
Grambling State ('Grambling State University', 100, 279)
Eastern Washington ('Eastern Washington University', 100, 949)
Montana ('The University of Montana', 100, 461)
Montana State ('Montana State University-Billings', 100, 459)
Idaho State ('Idaho State University', 100, 182)
Northern Arizona ('Northern Arizona University', 100, 22)
Cal Poly ('Molloy College', 55, 555)
Sacramento State ('California State University-Sacramento', 100, 51)
Southern Utah ('Southern Utah University', 100, 900)
North Dakota ('University of North Dakota', 100, 648)
Portland State ('Portland State University', 100, 705)
Weber State ('Weber State University', 100, 904)
Northern Colorado ('University of Northern Colorado', 100, 95)
UC Davis ('University of California-Davis', 77, 53)
Liberty ('West Liberty University', 100, 974)
Coastal Carolina ('Coastal Carolina University', 100, 822)
Presbyterian College ('Presbyterian College', 100, 819)
Charleston Sout

Yale ('Yale University', 100, 117)
Brown ('Brown University', 100, 796)
Princeton ('Princeton University', 100, 496)
Cornell ('Cornell University', 100, 524)
Columbia ('University of the District of Columbia', 100, 125)
North Carolina A ('North Carolina A & T State University', 100, 629)
Bethune-Cookman ('Bethune-Cookman University', 100, 132)
North Carolina Central ('North Carolina Central University', 100, 634)
South Carolina State ('South Carolina State University', 100, 823)
Hampton ('Hampton University', 100, 922)
Norfolk State ('Norfolk State University', 100, 931)
Morgan State ('Morgan State University', 100, 312)
Savannah State ('Savannah State University', 100, 173)
Florida A&M ('University of South Florida-Main Campus', 78, 150)
Delaware State ('Delaware State University', 100, 118)
Howard ('Howard University', 100, 129)
North Dakota State ('North Dakota State University-Main Campus', 100, 649)
Illinois State ('Illinois State University', 100, 202)
Western Illinois ('Western 

Make a list of school names with proper university names as seen in enrollment data. Fuzzwuzzy could not obtain correct matches

In [22]:
school_matches = [('Clemson', 'Clemson University'), ('Boston College', 'Boston College'), ('Florida State',
                  'Florida State University'), ('Wake Forest', 'Wake Forest University'), ('North Carolina St',
                  'North Carolina State University at Raleigh'), ('Maryland', 'University of Maryland-College Park'),
                  ('Georgia Tech', 'Georgia Institute of Technology-Main Campus'), ('Virginia Tech', 'Virginia Polytechnic Institute and State University'),
                  ('Miami', 'University of Miami'), ('North Carolina', 'University of North Carolina at Chapel Hill'),
                  ('Duke', 'Duke University'), ('Virginia', 'University of Virginia-Main Campus'), ('Nebraska', 'University of Nebraska-Lincoln'),
                  ('Missouri', 'University of Missouri-Columbia'), ('Kansas State', 'Kansas State University'),
                  ('Iowa State', 'Iowa State University'), ('Colorado', 'University of Colorado Boulder'),
                  ('Kansas', 'University of Kansas'), ('Texas', 'The University of Texas at Austin'),
                  ('Oklahoma State', 'Oklahoma State University-Main Campus'), ('Texas Tech', 'Texas Tech University'),
                  ('Oklahoma', 'University of Oklahoma-Norman Campus'), ('Texas A&M', 'Texas A & M University-College Station'),
                  ('Baylor', 'Baylor University'), ('Cincinnati', 'University of Cincinnati-Main Campus'), ('Pittsburgh', 'University of Pittsburgh-Pittsburgh Campus'),
                  ('West Virginia', 'West Virginia University'), ('Rutgers', 'Rutgers University-New Brunswick'), ('Connecticut', 'University of Connecticut'),
                  ('South Florida', 'University of South Florida-Main Campus'), ('Louisville', 'University of Louisville'),
                  ('Syracuse', 'Syracuse University'), ('Ohio State', 'Ohio State University-Main Campus'), ('Iowa', 'University of Iowa'),
                  ('Penn State', 'Pennsylvania State University-Main Campus'), ('Northwestern', 'Northwestern University'),
                  ('Wisconsin', 'University of Wisconsin-Madison'), ('Michigan State', 'Michigan State University'),
                  ('Purdue', 'Purdue University-Main Campus'), ('Minnesota', 'University of Minnesota-Twin Cities'),
                  ('Illinois', 'University of Illinois at Urbana-Champaign'), ('Michigan', 'University of Michigan-Ann Arbor'),
                  ('Indiana', 'Indiana University-Bloomington'), ('East Carolina', 'East Carolina University'), ('UCFUCF', 'University of Central Florida'),
                  ('Southern Mississippi', 'University of Southern Mississippi'), ('Marshall', 'Marshall University'),
                  ('UABUAB', 'University of Alabama at Birmingham'), ('Memphis', 'University of Memphis'), ('Houston', 'University of Houston'),
                  ('SMUSMU', 'Southern Methodist University'), ('Tulsa', 'University of Tulsa'), ('UTEPUTEP', 'The University of Texas at El Paso'),
                  ('Rice', 'Rice University'), ('Tulane', 'Tulane University of Louisiana'), ('Notre Dame', 'University of Notre Dame'),
                  ('Ohio', 'Ohio University-Main Campus'), ('Temple', 'Temple University'), ('Bowling Green', 'Bowling Green State University-Main Campus'),
                  ('Kent State', 'Kent State University at Kent'), ('Buffalo', 'University at Buffalo'), ('Akron', 'University of Akron Main Campus'),
                  ('Miami Oh', 'Miami University-Oxford'), ('Central Michigan', 'Central Michigan University'),
                  ('Northern Illinois', 'Northern Illinois University'), ('Western Michigan', 'Western Michigan University'),
                  ('Toledo', 'University of Toledo'), ('Ball State', 'Ball State University'), ('Eastern Michigan', 'Eastern Michigan University'),
                  ('TCUTCU', 'Texas Christian University'), ('BYUBYU', 'Brigham Young University-Provo'), ('Utah', 'University of Utah'),
                  ('Air Force', 'United States Air Force Academy'), ('Wyoming', 'University of Wyoming'), ('UNLVUNLV', 'University of Nevada-Las Vegas'),
                  ('San Diego State', 'San Diego State University'), ('New Mexico', 'University of New Mexico-Main Campus'),
                  ('Colorado State', 'Colorado State University-Fort Collins'), ('Oregon', 'University of Oregon'),
                  ('Arizona', 'University of Arizona'), ('Oregon State', 'Oregon State University'), ('Stanford', 'Stanford University'),
                  ('South Carolina', 'University of South Carolina-Columbia'), ('California', 'University of California-Berkeley'),
                  ('Washington', 'University of Washington-Seattle Campus'), ('UCLAUCLA', 'University of California-Los Angeles'),
                  ('Arizona State', 'Arizona State University-Tempe'), ('Washington State', 'Washington State University'),
                  ('Florida', 'University of Florida'), ('Georgia', 'University of Georgia'), ('Tennessee', 'The University of Tennessee-Knoxville'),
                  ('Kentucky', 'University of Kentucky'), ('South Carolina', 'University of South Carolina-Columbia'),
                  ('Vanderbilt', 'Vanderbilt University'), ('Alabama', 'The University of Alabama'), ('LSULSU', 'Louisiana State University and Agricultural & Mechanical College'),
                  ('Ole Miss', 'University of Mississippi'), ('Arkansas', 'University of Arkansas'), ('Auburn', 'Auburn University'),
                  ('Mississippi State', 'Mississippi State University'), ('Troy', 'Troy University'), ('Middle Tennessee', 'Middle Tennessee State University'),
                  ('Louisiana Monroe', 'University of Louisiana at Monroe'), ('Florida Atlantic', 'Florida Atlantic University'),
                  ('Louisiana', 'University of Louisiana at Lafayette'), ('Arkansas State', 'Arkansas State University-Main Campus'),
                  ('Florida Intl', 'Florida International University'), ('North Texas', 'University of North Texas'),
                  ('Western Kentucky', 'Western Kentucky University'), ('Boise State', 'Boise State University'),
                  ('Nevada', 'University of Nevada-Las Vegas'), ('Fresno State', 'California State University-Fresno'),
                  ('Idaho', 'University of Idaho'), ('Hawai', 'University of Hawaii at Manoa'), ('Louisiana Tech', 'Louisiana Tech University'),
                  ('Utah State', 'Utah State University'), ('New Mexico State', 'New Mexico State University-Main Campus'),
                  ('San Jose State', 'San Jose State University'), ('Florida State', 'Florida State University'),
                  ('Nicholls State', 'Nicholls State University'), ('Northwestern State', 'Northwestern State University of Louisiana'),
                  ('Alabama A&M', 'Alabama A & M University'), ('Alcorn State', 'Alcorn State University'), ('Jackson State', 'Jackson State University'),
                  ('Alabama State', 'Alabama State University'), ('Mississippi Valley State', 'Mississippi Valley State University'),
                  ('Prairie View A', 'Prairie View A & M University'), ('Texas Southern', 'Texas Southern University'),
                  ('Grambling State', 'Grambling State University'), ('Arkansas-Pine', 'University of Arkansas at Pine Bluff'),
                  ('Southern', 'Southern University and A & M College'), ('Montana State', 'Montana State University'),
                  ('Eastern Washington', 'Eastern Washington University'), ('Montana', 'The University of Montana'),
                  ('Sacramento State', 'California State University-Sacramento'), ('Weber State', 'Weber State University'),
                  ('Northern Arizona', 'Northern Arizona University'), ('Northern Colorado', 'University of Northern Colorado'),
                  ('Portland State', 'Portland State University'), ('Idaho State', 'Idaho State University'),
                  ('Liberty', 'Liberty University'), ('Coastal Carolina', 'Coastal Carolina University'), ('Stony Brook', 'Stony Brook University'),
                  ('Gardner-Webb', 'Gardner-Webb University'), ('VMIVMI', 'Virginia Military Institute'),
                  ('Charleston Southern', 'Charleston Southern University'), ('Presbyterian College', 'Presbyterian College'),
                  ('William', 'College of William and Mary'), ('Delaware', 'University of Delaware'), ('New Hampshire', 'University of New Hampshire-Main Campus'),
                  ('Villanova', 'Villanova University'), ('Massachusetts', 'University of Massachusetts-Amherst'),
                  ('Rhode Island', 'University of Rhode Island'), ('Richmond', 'University of Richmond'), ('Maine', 'University of Maine'),
                  ('James Madison', 'James Madison University'), ('Towson', 'Towson University'), ('Old Dominion', 'Old Dominion University'),
                  ('Georgia State', 'Georgia State University'), ('Southern Utah', 'Southern Utah University'), ('UC Davis', 'University of California-Davis'),
                  ('Cal Poly', 'California Polytechnic State University-San Luis Obispo'), ('South Dakota', 'University of South Dakota'),
                  ('North Dakota', 'University of North Dakota'), ('Penn', 'University of Pennsylvania'),
                  ('Harvard', 'Harvard University'), ('Yale', 'Yale University'), ('Brown', 'Brown University'),
                  ('Dartmouth', 'Dartmouth College'), ('Columbia', 'Columbia University in the City of New York'),
                  ('Cornell', 'Cornell University'), ('Princeton', 'Princeton University'), ('Bethune-Cookman', 'Bethune-Cookman University'),
                  ('South Carolina State', 'South Carolina State University'), ('Florida A&M', 'Florida Agricultural and Mechanical University'),
                  ('Hampton', 'Hampton University'), ('Norfolk State', 'Norfolk State University'), ('Savannah State', 'Savannah State University'),
                  ('Morgan State', 'Morgan State University'), ('Delaware State', 'Delaware State University'), ('North Carolina A', 'North Carolina A & T State University'),
                  ('North Carolina Central', 'North Carolina Central University'), ('Howard', 'Howard University'),
                  ('Northern Iowa', 'University of Northern Iowa'), ('Western Illinois', 'Western Illinois University'),
                  ('North Dakota State', 'North Dakota State University-Main Campus'), ('Illinois State', 'Illinois State University'),
                  ('Indiana State', 'Indiana State University'), ('Missouri State', 'Missouri State University-Springfield'),
                  ('South Dakota State', 'South Dakota State University'), ('Southern Illinois', 'Southern Illinois University-Carbondale'),
                  ('Youngstown State', 'Youngstown State University'), ('Central Connecticut', 'Central Connecticut State University'),
                  ('Robert Morris', 'Robert Morris University'), ('Duquesne', 'Duquesne University'), ('Bryant', 'Bryant University'),
                  ('Albany', 'SUNY at Albany'), ('Wagner', 'Wagner College'), ('Monmouth', 'Monmouth University'),
                  ('Sacred Heart', 'Sacred Heart University'), ('St Francis', 'St Francis College'), ('Southeast Missouri State', 'Southeast Missouri State University'),
                  ('Eastern Kentucky', 'Eastern Kentucky University'), ('Jacksonville State', 'Jacksonville State University'),
                  ('Murray State', 'Murray State University'), ('Tennessee-Martin', 'The University of Tennessee-Martin'),
                  ('Tennessee Tech', 'Tennessee Technological University'), ('Eastern Illinois', 'Eastern Illinois University'),
                  ('Austin Peay', 'Austin Peay State University'), ('Tennessee State', 'Tennessee State University'),
                  ('Lehigh', 'Lehigh University'), ('Colgate', 'Colgate University'), ('Holy Cross', 'College of the Holy Cross'),
                  ('Georgetown', 'Georgetown University'), ('Lafayette', 'University of Louisiana at Lafayette'),
                  ('Bucknell', 'Bucknell University'), ('Fordham', 'Fordham University'), ('Dayton', 'University of Dayton'),
                  ('Jacksonville', 'Jacksonville University'), ('Drake', 'Drake University'), ('San Diego', 'University of San Diego'),
                  ('Davidson', 'Davidson College'), ('Butler', 'Butler University'), ('Campbell', 'Campbell University'),
                  ('Marist', 'Marist College'), ('Valparaiso', 'Valparaiso University'), ('Wofford', 'Wofford College'),
                  ('Appalachian State', 'Appalachian State University'), ('Georgia Southern', 'Georgia Southern University'),
                  ('Chattanooga', 'The University of Tennessee-Chattanooga'), ('Elon', 'Elon University'), ('Furman', 'Furman University'),
                  ('Samford', 'Samford University'), ('Western Carolina', 'Western Carolina University'), ('The Citadel', 'Citadel Military College of South Carolina'),
                  ('Stephen F Austin', 'Stephen F Austin State University'), ('Mc', 'Mercer University'), ('Central Arkansas', 'University of Central Arkansas'),
                  ('Sam Houston State', 'Sam Houston State University'), ('Northwestern State', 'Northwestern State University of Louisiana'),
                  ('Nicholls State', 'Nicholls State University'), ('Texas State', 'Texas State University'), ('Southeastern Louisiana', 'Southeastern Louisiana University'),
                  ('Jackson State', 'Jackson State University'), ('South Alabama', 'University of South Alabama'),
                  ('Charlotte', 'University of North Carolina at Charlotte'), ('Hofstra', 'Hofstra University'), ('McNeese State', 'McNeese State University'),
                  ('Northeastern', 'Northeastern University'), ('UMass', 'University of Massachusetts-Amherst'), ('Winston-Salem', 'Winston-Salem State University'),
                  ('Abilene Christian', 'Abilene Christian University'), ('Incarnate Word', 'University of the Incarnate Word'),
                  ('Mercer', 'Mercer University'), ('Stetson', 'Stetson University'), ('East Tennessee State', 'East Tennessee State University')]



Convert list to a dictionary (easier for school look-ups)

In [24]:
school_matches_dict = {}

for match in school_matches:
    school_matches_dict[match[0]] = match[1]


In [25]:
combined_df[combined_df['TEAMS'] == 'Arkansas-Pine']

,TEAMS,WIN_PERCENT,YEARS,CAPACITY,CONFERENCE,AVG_ATTENDANCE
1117,Arkansas-Pine,0.454545,2009,72000,SEC,65112
1241,Arkansas-Pine,0.454545,2010,72000,SEC,68932
1364,Arkansas-Pine,0.545455,2011,72000,SEC,66990
1485,Arkansas-Pine,0.833333,2012,72000,SEC,68046
1613,Arkansas-Pine,0.181818,2013,72000,SEC,61596
1738,Arkansas-Pine,0.363636,2014,72000,SEC,66521
1864,Arkansas-Pine,0.181818,2015,72000,SEC,67326


In [ ]:
# Notes on fixes
# Navy, Army, Morehead State, Lamar, Houston Baptist, Kennessaw State have no match

In [26]:
team_check =  combined_df['TEAMS'].values
list_cross_check = [match[0] for match in school_matches]
for team in team_check:
    if team not in list_cross_check:
        print(team)

Navy
Army
Navy
Army
Navy
Army
Navy
Army
Navy
Army
Navy
Army
Navy
Army
Morehead State
Morehead State
Lamar
Morehead State
Lamar
Morehead State
Lamar
Houston Baptist
Morehead State
Lamar
Morehead State
Lamar
Houston Baptist
Kennesaw State
Morehead State
Lamar
Houston Baptist


### Work with merging enrollment data to the dataframe

In [27]:
college_enrollment_df.head()

,UnitId,Institution Name,Fall 2003,Fall 2004,Fall 2005,Fall 2006,Fall 2007,Fall 2008,Fall 2009,Fall 2010,Fall 2011,Fall 2012,Fall 2013,Fall 2014,Fall 2015,Unnamed: 15
0,100654,Alabama A & M University,6588,6323,6182,5706,5706,5124,5327,5814,4922,4853,5020,5333,5628,NaN
1,100663,University of Alabama at Birmingham,16357,16693,16572,16246,16246,16149,16874,17543,17575,17999,18568,18698,18333,NaN
2,100706,University of Alabama in Huntsville,7051,7036,7084,7264,7264,7431,7681,7614,7629,7636,7376,7348,7866,NaN
3,100724,Alabama State University,6024,5653,5469,5608,5608,5695,5564,5705,5425,5816,6075,5519,5383,NaN
4,100751,The University of Alabama,20290,20929,21793,25544,25544,27014,28699,30127,31647,33503,34752,36047,37098,NaN


Select the correct years and change column names

In [28]:
college_enrollment_df = college_enrollment_df[['Institution Name', 'Fall 2009', 'Fall 2010', 'Fall 2011', 'Fall 2012', 'Fall 2013', 'Fall 2014', 'Fall 2015']]

In [29]:
college_enrollment_df.columns = ['Institution Name', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
college_enrollment_df.head()

,Institution Name,2009,2010,2011,2012,2013,2014,2015
0,Alabama A & M University,5327,5814,4922,4853,5020,5333,5628
1,University of Alabama at Birmingham,16874,17543,17575,17999,18568,18698,18333
2,University of Alabama in Huntsville,7681,7614,7629,7636,7376,7348,7866
3,Alabama State University,5564,5705,5425,5816,6075,5519,5383
4,The University of Alabama,28699,30127,31647,33503,34752,36047,37098


Make a list of teams and years to obtain enrollment data for all schools in previous dataframe

In [30]:
teams_in_combined_df = combined_df['TEAMS'].values
years_in_combined_df = combined_df['YEARS'].values
team_year_combo = list(zip(teams_in_combined_df, years_in_combined_df))
enrollment_values = []
for entry in team_year_combo:
    if entry[0] in school_matches_dict:
        school_name = school_matches_dict[entry[0]]
        a = college_enrollment_df[college_enrollment_df['Institution Name'] == school_name][entry[1]].values
        #print(entry[0], entry[1], school_name, a[0])
    else:
        #print('NOT FOUND:   ' + entry[0])
        a = [np.nan]
    enrollment_values.append(a[0])
    
enrollment_values

    

[19111,
 15036,
 39785,
 7079,
 33819,
 37195,
 20291,
 30870,
 15629,
 28916,
 14350,
 24355,
 24100,
 31237,
 23581,
 27945,
 33010,
 29242,
 50995,
 23033,
 30049,
 25881,
 48702,
 14614,
 31134,
 28328,
 28898,
 37366,
 25029,
 40022,
 21016,
 19638,
 55014,
 28987,
 45185,
 19853,
 41654,
 47071,
 41052,
 51659,
 43881,
 41674,
 42347,
 27654,
 53401,
 15300,
 13776,
 16874,
 21424,
 37000,
 10891,
 4187,
 21011,
 5576,
 11464,
 nan,
 11816,
 nan,
 22647,
 36507,
 17309,
 25127,
 28881,
 25959,
 16884,
 27247,
 24424,
 24576,
 23064,
 21401,
 22893,
 8853,
 34130,
 29284,
 4620,
 12427,
 29080,
 32817,
 27241,
 28902,
 22335,
 38767,
 21950,
 18498,
 28482,
 35830,
 45943,
 38550,
 68064,
 26101,
 50691,
 34885,
 29934,
 26295,
 28482,
 12506,
 28699,
 28643,
 15932,
 19849,
 24602,
 18601,
 29328,
 25188,
 9004,
 27637,
 16361,
 12156,
 39610,
 35003,
 20712,
 18933,
 29080,
 21500,
 11957,
 20435,
 11264,
 15612,
 18526,
 31280,
 40416,
 37641,
 34376,
 14868,
 19453,
 7162,
 31

In [ ]:
# Notes for changes
# Arkansas-Pine
# North Carolina St - North Carolina A & T State University = should be North Carolina State University at Raleigh
# Maryland - Loyola University Maryland = should be University of Maryland-College Park
# North Carolina - North Carolina A & T State University
# Virginia ('Virginia Polytechnic Institute and State University'

In [31]:
college_enrollment_df[college_enrollment_df['Institution Name'].str.contains('Arkansas')]

,Institution Name,2009,2010,2011,2012,2013,2014,2015
23,University of Arkansas at Little Rock,13132,13176,13068,12872,12377,11645,11891
24,University of Arkansas,19849,21405,23199,24537,25341,26237,26754
25,University of Arkansas at Pine Bluff,3792,3428,3188,2828,2615,2513,2658
26,Arkansas State University-Main Campus,12156,13415,13900,13877,13552,13144,13410
27,Arkansas Tech University,8814,9815,10464,10950,11369,12002,12054
28,University of Arkansas at Monticello,3479,3638,3920,3945,3893,3854,3643
29,University of Central Arkansas,11781,11444,11163,11107,11534,11698,11754
35,Southern Arkansas University Main Campus,3226,3379,3382,3330,3404,3546,4095


Add final enrollment column to the original dataframe to merge all data together.

In [32]:
combined_df['Enrollment'] = enrollment_values

In [33]:
combined_df.head()

,TEAMS,WIN_PERCENT,YEARS,CAPACITY,CONFERENCE,AVG_ATTENDANCE,Enrollment
0,Clemson,0.642857,2009,81500,ACC,75793,19111
1,Boston College,0.615385,2009,44500,ACC,35716,15036
2,Florida State,0.538462,2009,79560,ACC,74345,39785
3,Wake Forest,0.416667,2009,31500,ACC,31791,7079
4,North Carolina St,0.416667,2009,57583,ACC,56422,33819


In [34]:
# ADD SOME MISSING ENTRIES

combined_df.at[851, 'AVG_ATTENDANCE'] = 102004
combined_df.at[1223, 'AVG_ATTENDANCE'] = 11445
combined_df.at[1348, 'AVG_ATTENDANCE'] = 12793
combined_df.at[1468, 'AVG_ATTENDANCE'] = 13574
combined_df.at[1593, 'AVG_ATTENDANCE'] = 13155
combined_df.at[1715, 'AVG_ATTENDANCE'] = 9505
combined_df.at[1837, 'AVG_ATTENDANCE'] = 10678
combined_df.at[1484, 'AVG_ATTENDANCE'] = 12241
combined_df.at[1607, 'AVG_ATTENDANCE'] = 4130
combined_df.at[1730, 'AVG_ATTENDANCE'] = 12268
combined_df.at[1855, 'AVG_ATTENDANCE'] = 12343

In [38]:
combined_df['AVG_ATTENDANCE'] = pd.to_numeric(combined_df['AVG_ATTENDANCE'], errors='coerce')
combined_df['CAPACITY'] = pd.to_numeric(combined_df['CAPACITY'], errors='coerce')
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1732 entries, 0 to 1864
Data columns (total 7 columns):
TEAMS             1732 non-null object
WIN_PERCENT       1732 non-null float64
YEARS             1732 non-null object
CAPACITY          1732 non-null int64
CONFERENCE        1732 non-null object
AVG_ATTENDANCE    1693 non-null float64
Enrollment        1701 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 188.2+ KB


In [39]:
combined_df.head()

,TEAMS,WIN_PERCENT,YEARS,CAPACITY,CONFERENCE,AVG_ATTENDANCE,Enrollment
0,Clemson,0.642857,2009,81500,ACC,75793.0,19111
1,Boston College,0.615385,2009,44500,ACC,35716.0,15036
2,Florida State,0.538462,2009,79560,ACC,74345.0,39785
3,Wake Forest,0.416667,2009,31500,ACC,31791.0,7079
4,North Carolina St,0.416667,2009,57583,ACC,56422.0,33819


Make a column of capacity of stadiums (in percent) using the attendance divided by the capacities

In [40]:
combined_df['PERC_CAPACITY'] = combined_df['AVG_ATTENDANCE'] / combined_df['CAPACITY']

In [41]:
# FIX ENTRIES THAT WERE MISLABELED

# 108 attend should be 10204 DONE
# 222 attend should be 19808 DONE
# Ole Miss capacity should be 64038
# 346 attend 18411 DONE
# 474 attend should be 13634 DONE
# 423 attend nan DONE
# 535 attend 15453 DONE
# 556 attend 15830 
# 664 attend 11966
# 680 attend 16088
# 792 attend 15381
# 810 attend 11124
# 1013 attend 10887
# 1019 Delaware cap should be 22000
# 1023 North Carolina Central cap 62000
# 1024 attend none
# 1301 south dakota state cap should be 19340
# 1180 Central Connecticut cap should be 40000
# 1395 delaware capacity should be 22000
# 1847 attend should be nan
# 1851 attend should be nan
# 1772 attend should be nan
# 1727 attend shoyld be nan
# 1726 attend nan
# 1551 attend nan
# 1521 attena nan
# 1520 attend nan
# 1424 attend nan
# 1265 attend should be 10005
# 1139 attend 13005
combined_df.at[108, 'AVG_ATTENDANCE'] = 10204
combined_df.at[222, 'AVG_ATTENDANCE'] = 19808
combined_df.at[346, 'AVG_ATTENDANCE'] = 18411
combined_df.at[474, 'AVG_ATTENDANCE'] = 13634
combined_df.at[423, 'AVG_ATTENDANCE'] = np.nan
combined_df.at[535, 'AVG_ATTENDANCE'] = 15453
combined_df.at[556, 'AVG_ATTENDANCE'] = 15830
combined_df.at[664, 'AVG_ATTENDANCE'] = 11966
combined_df.at[680, 'AVG_ATTENDANCE'] = 16088
combined_df.at[792, 'AVG_ATTENDANCE'] = 15381
combined_df.at[810, 'AVG_ATTENDANCE'] = 11124
combined_df.at[1013, 'AVG_ATTENDANCE'] = 10887
combined_df.at[1024, 'AVG_ATTENDANCE'] = np.nan
combined_df.at[1847, 'AVG_ATTENDANCE'] = np.nan
combined_df.at[1851, 'AVG_ATTENDANCE'] = np.nan
combined_df.at[1772, 'AVG_ATTENDANCE'] = np.nan
combined_df.at[1727, 'AVG_ATTENDANCE'] = np.nan
combined_df.at[1726, 'AVG_ATTENDANCE'] = np.nan
combined_df.at[1551, 'AVG_ATTENDANCE'] = np.nan
combined_df.at[1521, 'AVG_ATTENDANCE'] = np.nan
combined_df.at[1520, 'AVG_ATTENDANCE'] = np.nan
combined_df.at[1424, 'AVG_ATTENDANCE'] = np.nan
combined_df.at[1265, 'AVG_ATTENDANCE'] = 10005
combined_df.at[1139, 'AVG_ATTENDANCE'] = 13005
combined_df.at[1171, 'AVG_ATTENDANCE'] = np.nan
combined_df.at[1298, 'AVG_ATTENDANCE'] = np.nan

In [42]:
# CHANGE CAPACITIES THAT WERE ENTERED INCORRECTLY FROM SCRAPING (MULTIPLE STADIUM OPTIONS FOR SOME TEAMS
# DUE TO NAMES)

combined_df.loc[(combined_df['TEAMS'] == 'Delaware'), 'CAPACITY'] = 22000
combined_df.loc[(combined_df['TEAMS'] == 'Ole Miss'), 'CAPACITY'] = 64038
combined_df.loc[(combined_df['TEAMS'] == 'North Carolina Central'), 'CAPACITY'] = 62000
combined_df.loc[(combined_df['TEAMS'] == 'South Dakota State'), 'CAPACITY'] = 19340
combined_df.loc[(combined_df['TEAMS'] == 'Central Connecticut'), 'CAPACITY'] = 40000

In [43]:
combined_df['PERC_CAPACITY'] = combined_df['AVG_ATTENDANCE'] / combined_df['CAPACITY']

In [44]:
combined_df[combined_df['TEAMS'] == 'Delaware']

,TEAMS,WIN_PERCENT,YEARS,CAPACITY,CONFERENCE,AVG_ATTENDANCE,Enrollment,PERC_CAPACITY
1019,Delaware,0.545455,2009,22000,MEAC,20750.0,21138,0.943182
1136,Delaware,0.800000,2010,22000,MEAC,17753.0,21177,0.806955
1264,Delaware,0.636364,2011,22000,MEAC,19019.0,21489,0.864500
1395,Delaware,0.454545,2012,22000,MEAC,18542.0,21856,0.842818
1515,Delaware,0.583333,2013,22000,MEAC,18108.0,22166,0.823091
1641,Delaware,0.500000,2014,22000,MEAC,15682.0,22680,0.712818
1767,Delaware,0.363636,2015,22000,MEAC,15826.0,22852,0.719364


### Check to make sure percent capacities are all reasonable (not greater than ~120%)

In [45]:
combined_df[combined_df['PERC_CAPACITY'] >= 1.20]

,TEAMS,WIN_PERCENT,YEARS,CAPACITY,CONFERENCE,AVG_ATTENDANCE,Enrollment,PERC_CAPACITY
1806,Bryant,0.454545,2015,4400,Northeast,5493.0,3670,1.248409


### Make a final dataframe of all merged and cleaned data for pickling

In [46]:
NCAA_df = combined_df[['TEAMS', 'WIN_PERCENT', 'YEARS', 'CONFERENCE', 'Enrollment', 'PERC_CAPACITY']]
NCAA_df.columns = ['TEAM', 'WIN_PERCENTAGE', 'YEAR', 'CONFERENCE', 'ENROLLMENT', 'PERCENT_CAPACITY']
NCAA_df.head()

,TEAM,WIN_PERCENTAGE,YEAR,CONFERENCE,ENROLLMENT,PERCENT_CAPACITY
0,Clemson,0.642857,2009,ACC,19111,0.929975
1,Boston College,0.615385,2009,ACC,15036,0.802607
2,Florida State,0.538462,2009,ACC,39785,0.934452
3,Wake Forest,0.416667,2009,ACC,7079,1.009238
4,North Carolina St,0.416667,2009,ACC,33819,0.979838


In [47]:
NCAA_df[NCAA_df['PERCENT_CAPACITY'] >= 1.20]

,TEAM,WIN_PERCENTAGE,YEAR,CONFERENCE,ENROLLMENT,PERCENT_CAPACITY
1806,Bryant,0.454545,2015,Northeast,3670,1.248409


In [ ]:
NCAAdf_path = '/Users/murdock/Documents/PDF_files/'
NCAA_df.to_pickle(NCAAdf_path+ 'ncaa_df.pkl')